In [1]:
from torch import empty

In [2]:
empty(10,10).shape

torch.Size([10, 10])

In [3]:
class Module ( object ):
    
    def param ( self ):
        return [0]

In [4]:
class LossMSE(Module):
    
    def __init__(self, prev_module = None):

        self.prev_module =  prev_module
        
    def set_truth(self,y_true):
        self.y_true = y_true
        
    def forward (self , input_ ):
        assert input_.shape[1] == self.y_true.shape[1], "Input and output size must match!"
        self.curr_input = input_
        return (self.y_true-input_).square().mean()
        
    def backward (self):
        #Calculate gradient
        grad = -2 *(self.y_true-self.curr_input) / (self.curr_input.shape[1])        
        
        #Call backward() for previous module
        if self.prev_module is not None:
            prev_grads = self.prev_module.backward(grad)
    
    def param ( self ):
        return []
    

In [5]:
class ReLU(Module):
    
    def __init__(self, prev_module = None):
        self.prev_module =  prev_module
        self.curr_grad = 0 #Temporary

    def forward (self , input_ ):
        self.curr_grad = (input_ > 0)
        return input_ * self.curr_grad
        
    def backward (self , gradwrtoutput):
        #Calculate gradient
        grad = self.curr_grad * gradwrtoutput
        
        #Call backward() for previous module
        if self.prev_module is not None:
            prev_grads = self.prev_module.backward(grad)
    
    def param ( self ):
        return []

In [68]:
class FCC(Module):
    
    def __init__(self, input_size, output_size, prev_module = None):
        self.input_size = input_size
        self.output_size = output_size
        self.prev_module =  prev_module
        self.weights = empty(input_size,output_size).fill_(0.1) #TODO better init
        self.bias = empty(1,output_size).fill_(0.1) #TODO better init
        self.curr_input = 0

    def forward (self , input_ ):
        assert input_.shape[1] == self.input_size, "Input size must match!" 
        #print("input size:", input_.shape)
        #print("self.weights size:", self.weights.shape)
        out = input_ @ (self.weights) 
        #print("biass size:", self.bias.shape)
        #print("out size:", out.shape)
        out += self.bias
        assert out.shape[1] == self.output_size, "Output size must match!" 
        self.curr_input = input_
        return out
        
    def backward (self , gradwrtoutput):
        #Calculate gradient
        grad = gradwrtoutput.multiply(self.weights)
        
        #update weights
        self.update(gradwrtoutput,0.05) #TODO learning rate
        
        #Call backward() for previous module
        if self.prev_module is not None:
            prev_grads = self.prev_module.backward(grad)
    
    def update(self,gradwrtoutput,learning_rate):
        self.weights -= learning_rate * ((self.curr_input.T) @ gradwrtoutput )
        self.bias -= learning_rate * gradwrtoutput
        
    def param ( self ):
        return [self.weights, self.bias]
    

In [69]:
class NN_builder():
    
    def __init__(self):
        self.layers = []
        layer0 = FCC(2,1)
        self.layers.append(layer0)
        layer1 = ReLU(layer0)
        self.layers.append(layer1)
        layer2 = LossMSE(layer1)
        self.layers.append(layer2)
    
    def model_train(self,input_, g_truth):
        curr = input_
        self.layers[-1].set_truth(g_truth)
        for layer in self.layers:
            curr = layer.forward(curr)
        self.layers[-1].backward()        
        
    def model_eval(self,input_):
        curr = input_
        for layer in self.layers[:-1]:
            curr = layer.forward(curr)
            #print(curr)
        return curr
    

In [8]:
test = empty(1,2).fill_(5)
truth = empty(1,1).fill_(10)

In [9]:
truth.shape

torch.Size([1, 1])

In [10]:
builder = NN_builder()

In [11]:
out = builder.model_train(test,truth)

### These two are just for a quick check I know they are terrible :D

In [48]:
def stupid_test_function(a,b):
    if a == 0 or b == 0:
        return 0
    else:
        return 1

In [83]:
def stupid_acc_func(pred,true):
    pred = pred.item() > 0.9
    return (pred == true.item())
    

In [84]:
builder = NN_builder()
test = empty(1000,2).random_(0,2)
for i in range(1000):
    truth = empty(1,1).fill_(stupid_test_function(test[i,0],test[i,1]))
    inp = test[i,:].unsqueeze(0)
    out = builder.model_train(inp,truth)
    
test2 = empty(100,2).random_(0,2)
acc = 0
count = 0
for i in range(100):
    truth = empty(1,1).fill_(stupid_test_function(test2[i,0],test2[i,1]))
    inp = test2[i,:].unsqueeze(0)
    out = builder.model_eval(inp)
    if out.item() != 0:
        count = count + 1
    if stupid_acc_func(out,truth):
        acc = acc + 1
acc /100

1.0

In [82]:
count

71

In [ ]:
    print("out:", out.item() )
    print("truth:", truth.item() )